# InceptionV3 Cancer Classification

In [ ]:
# Import some packages to use
import cv2 
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
%matplotlib inline 

import os 
import random 
import gc 

#### Image set preparation

In [ ]:
print(os.getcwd())

In [ ]:
# Count Negative Images
negative_path = '/home/ssj0921/SSJ/Data/20190731_Classification/TCIA/TCGA_Normal_256/' 
negative_list = []
for (path, dir, files) in os.walk(negative_path):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.jpg':
            filePath = "{0}/{1}".format(path, filename)
            # print(filePath)
            negative_list.append(filePath)  

# Create Data Frame
# Negative = 0
negative_df = pd.DataFrame(data={'filename': negative_list, 
                               'label': [0 for i in range(len(negative_list))]})

len(negative_list), len(negative_df)

In [ ]:
# Count Positive Images
positive_path = '/home/ssj0921/SSJ/Data/20190731_Classification/TCIA/TCGA_Cancer_256/' 
positive_list = []
for (path, dir, files) in os.walk(positive_path):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.jpg':
            filePath = "{0}/{1}".format(path, filename)
            positive_list.append(filePath)

# Create Data Frame
# Positive = 1
positive_df = pd.DataFrame(data={'filename': positive_list, 
                               'label': [1 for i in range(len(positive_list))]})

len(positive_list), len(positive_df)

In [ ]:
# Merge Cancer + Normal ImageList
train_img = negative_list + positive_list
roi_img_df = negative_df.append(positive_df)

# Shuffle
sh_df = roi_img_df.sample(frac=1).reset_index(drop=True)

len(train_img), len(roi_img_df), len(sh_df)

In [ ]:
len(sh_df)

In [ ]:
sh_df.head()

In [ ]:
# Matplotlib을 이용하여 Image 읽기
from matplotlib.image import imread

im_list = []

for path in sh_df['filename']:
    img = imread(path)
    im_list.append(img)

In [ ]:
import matplotlib.pyplot as plt

def showImage(img):
    plt.imshow(img)    
    plt.show()

# Draw Image
showImage(im_list[0])

In [ ]:
# Seperate Color Image View
def showImages(input_img):
    f, axarr = plt.subplots(2,2)

    axarr[0,0].imshow(input_img[...,0]) # R
    axarr[0,1].imshow(input_img[...,1]) # G
    axarr[1,0].imshow(input_img[...,2]) # B
    
    return

showImages(im_list[0])

In [ ]:
import numpy as np

x = np.array(im_list)
y = np.array(sh_df['label'])

#### Image visualization

In [ ]:
# Visualization
def visualize_data(cancer_images, normal_images, N_TO_VISUALIZE):
    # INPUTS
    # positive_images - Images where the label = 1 (True)
    # negative_images - Images where the label = 0 (False)
 
    figure = plt.figure()
    count = 0
 
    for i in range(cancer_images.shape[0]):
        count += 1
        figure.add_subplot(2, cancer_images.shape[0], count)
        plt.imshow(cancer_images[i, :, :])
        plt.axis('off')
        plt.title("Positive".format(i))
 
        figure.add_subplot(1, normal_images.shape[0], count)
        plt.imshow(normal_images[i, :, :])
        plt.axis('off')
        plt.title("Negative".format(i), y=-0.3)
        
    plt.show()

In [ ]:
# Number of positive and negative examples to show
N_TO_VISUALIZE = 5
 
# Select the first N Cancer examples
positive_example_indices = (y == 1)
positive_examples = x[positive_example_indices, :, :]
positive_examples = positive_examples[0:N_TO_VISUALIZE, :, :]
 
# Select the first N Normal examples
negative_example_indices = (y == 0)
negative_examples = x[negative_example_indices, :, :]
negative_examples = negative_examples[0:N_TO_VISUALIZE, :, :]
 
# Call the visualization function
visualize_data(positive_examples, negative_examples, N_TO_VISUALIZE)

### [3] Image resize and data set split

In [ ]:
from PIL import Image

# Resize Image
train = []

for img in x:
    arr = Image.fromarray(img)
    res = arr.resize((299, 299))
    train.append(np.array(res))
    
train = np.array(train)

In [ ]:
from sklearn.model_selection import train_test_split

# Split Data
x_train, x_test, y_train, y_test = train_test_split(train, y, test_size=0.2)

In [ ]:
# Generalize 
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

In [ ]:
print('training dataset =', len(x_train))
print('training label =', len(y_train))

print('test dataset =', len(x_test))
print('test label =', len(y_test))

## Call a Inception pretrained model

In [ ]:
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
import keras
from keras.applications import InceptionV3
from keras import layers
from keras import models
from keras import regularizers
from keras import optimizers

from keras.models import Sequential
from keras.layers import Dense, Activation

In [ ]:
# 세션을 gpu의 메모리로 처리하도록 설정
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
# Now when creating your session pass this config to it.
sess = tf.Session(config=config)

In [ ]:
strategy = tf.distribute.MirroredStrategy()
run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)

with strategy.scope():
    
    conv_base = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
    
    model = Sequential()
    model.add(conv_base)
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    conv_base.trainable = True
    
    model.compile(loss='binary_crossentropy', 
                  optimizer=optimizers.RMSprop(lr=2e-5), metrics=['acc'], options = run_opts)
    
    #model.summary()

In [ ]:
print('Number of trainable weights before freezing the conv base:', len(model.non_trainable_weights))
print('Number of trainable weights after freezing the conv base:', len(model.trainable_weights))
len(x_train), len(y_train)

In [ ]:
from livelossplot.keras import PlotLossesCallback
plot_losses_call_back = PlotLossesCallback()

hist = model.fit(x_train, y_train, epochs=5, batch_size=10, 
                 validation_split=0.2, verbose=0,                 
                 callbacks=[plot_losses_call_back])

In [ ]:
fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

plt.title('Training Process', fontsize=15)

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')

acc_ax.plot(hist.history['acc'], 'b', label='train acc')
acc_ax.plot(hist.history['val_acc'], 'g', label='val acc')

loss_ax.set_xlabel('epoch', fontsize=11)
loss_ax.set_ylabel('loss', fontsize=11)
acc_ax.set_ylabel('accuray', fontsize=11)

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

fig = plt.gcf()
plt.show()
#fig.savefig('Class_InceptionV3_20190803.png')

In [ ]:
# Test Data Set의 Normal/Cancer 비율 확인하기
Normal = 0
Cancer = 0

for i in y_test:
    if i == 1.0:
        Normal += 1
    else:
        Cancer += 1
        
print('Normal:', Normal, ', Cancer:', Cancer)

In [ ]:
# 테스트 데이터로 평가하기
score = model.evaluate(x_test, y_test, verbose=1)
print('loss=', score[0])
print('accuracy=', score[1])

In [ ]:
from sklearn.metrics import roc_auc_score, auc, roc_curve

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()

y_score = model.predict_proba(x_test)
n_classes = y_test.shape[0]

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

lw = 2
class_F = y_test
proba_F = y_score

false_positive_rate_F, true_positive_rate_F, thresholds_F = roc_curve(class_F, proba_F)
roc_auc_F = auc(false_positive_rate_F, true_positive_rate_F)

plt.title('Cancer Classification AUROC\n(trained by TCIA)\n(Internal Validation with TICA)', fontsize=15)
plt.xlabel('False Positive Rate(1 - Specificity)', fontsize=13)
plt.ylabel('True Positive Rate(Sensitivity)', fontsize=13)

plt.plot(false_positive_rate_F, true_positive_rate_F, 'darkorange', 
         label='ROC curve (AUC = %0.5f)'% roc_auc_F)

plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--', label='Random guess')

plt.legend(loc='lower right')
plt.show()
#fig.savefig('Cancer_AUROC_InceptionV3_E50_TCGA_20191118.png')

In [ ]:
os.chdir('/home/ssj0921/SSJ/Model/')

In [ ]:
os.listdir()

In [ ]:
# 모델 저장하기
from keras.models import load_model
model.save('Model_Class_256_InceptionV3_E30_TCIA_20191101.h5')